In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
from pixell import enmap, enplot, reproject
import healpy as hp
import warnings
warnings.filterwarnings('ignore')
from pixell import curvedsky
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
%matplotlib inline
from astropy.io import fits
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u
import astropy.coordinates as coord
import error_analysis_funcs as ef

[1687286948.906724] [kingcrab:3821838:0]        ib_iface.c:1035 UCX  ERROR ibv_create_cq(cqe=4096) failed: Cannot allocate memory


[kingcrab:3821838] pml_ucx.c:309  Error: Failed to create UCP worker


In [2]:
def eshow(x,**kwargs):
    ''' Define a function to help us plot the maps neatly '''
    plots = enplot.get_plots(x, **kwargs)
    enplot.show(plots, method = "ipython")

In [3]:
# Read in the ymap
ymap = enmap.read_map("/mnt/raid-cita/mlokken/data/act_ymaps/ilc_SZ_deproj_cib_yy.fits")
mask_em = enmap.read_map("/mnt/raid-cita/mlokken/data/masks/outputMask_wide_mask_GAL070_apod_1.50_deg_wExtended.fits")
# noisemap = hp.read_map('/mnt/raid-cita/mlokken/data/act_ymaps/sim_ymaps/wsky_t_nilc_y_220302_masks_20200723_bin_apod_cal_True_dg1_w4.0_h4.0_lsmooth400_lmax10799_set0_0000.fits')
# noisemask = hp.read_map("/mnt/raid-cita/mlokken/data/act_ymaps/wide_mask_GAL080_apod_3.00_deg_4096_hpx.fits")
# We smooth with healpy in order to use a top hat smoothing function
# set 0 to nan in the mask - this helps with the std later
# mask[np.where(mask < 0.1)] = np.nan
mask_em[np.where(mask_em == 0)] = np.nan

In [ ]:
eshow(mask_em, downgrade=30)

In [12]:
# input a list of scales ranging from the bin size (5 Mpc) at z_min ~ 0.2 to z_max ~ 0.9
z_min, z_max = 0.2, 0.9
binsize = 5*u.Mpc
theta_max = cosmo.arcsec_per_kpc_comoving(z_min).to(u.deg/u.Mpc)*binsize
theta_min = cosmo.arcsec_per_kpc_comoving(z_max).to(u.deg/u.Mpc)*binsize
scale_list = np.linspace(theta_min.value, theta_max.value, 4)
print(scale_list)

[0.09124623 0.17403076 0.25681529 0.33959981]


In [13]:
ell_max = np.pi*u.rad/theta_min.to(u.rad)
ell_min = np.pi*u.rad/theta_max.to(u.rad)
print(ell_min, ell_max)

530.035626315731 1972.6842033176597


In [5]:
with fits.open("/mnt/raid-cita/mlokken/data/cluster_cats/redmapper2.2.1_lgt20vl50_mask_actshr1deg_des_cutpt8.fit") as stackpoints_file:
    ra = stackpoints_file[1].data['RA']
    dec = stackpoints_file[1].data['dec']
    lamda = stackpoints_file[1].data['lambda_chisq']
ra = ra[lamda>20]
wrap = ra>180
ra[wrap]=ra[wrap]-360
dec = dec[lamda>20]
region_ids       = np.loadtxt("/home/mlokken/oriented_stacking/general_code/labels_24_regions_ACTxDES_lambdagt20.txt", dtype=int)

In [11]:
# define the minmax list for the regions
minmax_list = []
reglist = []
for i in range(np.amax(region_ids[:,1])+1):
    ra_max  = np.amax(ra[region_ids[:,1]==i])
    dec_max = np.amax(dec[region_ids[:,1]==i])
    ra_min  = np.amin(ra[region_ids[:,1]==i])
    dec_min = np.amin(dec[region_ids[:,1]==i])
    minmax_list.append([ra_min, ra_max, dec_min, dec_max])
    reglist.append(i)

## See what the results are for the real y map

In [10]:
vars_with_scale_y = []
var_maps_y = []
for theta in scale_list:
    print(theta)
    ymap_filt = ef.tophat_smooth_pixell(ymap, theta, is_enmap=True)
    print("map filtered.")
    variances, var_map = ef.spatial_weights(ymap_filt, mask_em, minmax_list=minmax_list, is_enmap=True)
    var_maps_y.append(var_map)
    vars_with_scale_y.append(variances)

0.09124623175735683
map filtered.
0.1740307591022704
map filtered.
0.25681528644718393
map filtered.
0.33959981379209747
map filtered.


# And now the noise map

In [ ]:
# vars_with_scale_noise = []
# var_maps_noise = []
# for theta in scale_list:
#     print(theta)
#     ymap_filt = ef.tophat_smooth_pixell(noisemap, theta, is_enmap=False)
#     print("map filtered.")
#     variances, var_map = ef.spatial_weights(ymap_filt, noisemask, minmax_list=minmax_list, is_enmap=False)
#     var_maps_noise.append(var_map)
#     vars_with_scale_noise.append(variances)
#     break

In [12]:
weights_y = 1/np.average(np.asarray(vars_with_scale_y), axis=0)
weights_y /=np.average(weights_y) # normalize again
# weights_noise = 1/np.average(np.asarray(vars_with_scale_noise), axis=0)

In [ ]:
plt.plot(np.arange(len(weights_y)), weights_y, label='from y map')
# plt.plot(np.arange(len(weights_noise)), weights_noise, label='from noise map')
plt.legend()

In [13]:
np.average(weights_y)

1.0

In [14]:
vmap = eshow(np.average(var_maps_y, axis=0), **{"colorbar":True, "downgrade":20})
# vmap.draw_colorbar()

KeyboardInterrupt: 

In [ ]:
# hp.mollview(np.average(var_maps_noise, axis=0))

In [ ]:
cmap = plt.cm.get_cmap('prism',60)
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection="mollweide")
for i in range(48):
    in_reg = region_ids[:,1] == i
    racoord = coord.Angle(ra[in_reg]*u.degree)
    racoord = racoord.wrap_at(180*u.degree)
    dec_coord = coord.Angle(dec[in_reg]*u.degree)
    ax.scatter(racoord.radian,dec_coord.radian,color=cmap(i), s=1)
ax.set_xticklabels(['14h','16h','18h','20h','22h','0h','2h','4h','6h','8h','10h'])
ax.grid(True)
ax.set_xlabel("RA")
ax.set_ylabel("Dec")

In [15]:
# stack the region id and weights list side-by-side
save_list = np.column_stack((reglist, weights_y))
save_list

array([[ 0.        ,  1.16897049],
       [ 1.        ,  1.17289996],
       [ 2.        ,  1.15404387],
       [ 3.        ,  0.94678903],
       [ 4.        ,  0.70193131],
       [ 5.        ,  0.86712162],
       [ 6.        ,  0.69945122],
       [ 7.        ,  0.70337117],
       [ 8.        ,  1.32533075],
       [ 9.        ,  1.22003835],
       [10.        ,  0.96375283],
       [11.        ,  1.15751372],
       [12.        ,  0.62127485],
       [13.        ,  0.85148169],
       [14.        ,  0.60666126],
       [15.        ,  1.04055235],
       [16.        ,  1.12119695],
       [17.        ,  1.0922598 ],
       [18.        ,  1.11450928],
       [19.        ,  0.95949886],
       [20.        ,  1.18822865],
       [21.        ,  1.00756369],
       [22.        ,  1.13247638],
       [23.        ,  1.18308192]])

In [16]:
np.savetxt('weights_24_regions_ACT_ilc_SZ_deproj_cib_yy_DES_redmapper_lambdagt20.txt', save_list)

In [ ]:
stop

In [ ]:
# For some reason these give slightly different maps but reasonably similar variances. Not totally sure why
# the pixell routine is faster

def get_window(scale):
    theta = np.linspace(0, np.deg2rad(scale))
    beam  = np.ones(len(theta))
    beam = hp.beam2bl(beam, theta, nside*2)
    return beam

def smooth_map_pixell(scale):
    beam = get_window(scale)
    ymap_filt = curvedsky.filter(ymap, beam, lmax = nside*3)
    return ymap_filt

def smooth_map(scale):
    ''' Smooths map with a certain scale using a top hat. Scale is set in degrees.'''

    beam = get_window(scale)

    # Smooth the map
    smoothed_hp = hp.sphtfunc.smoothing(ymap_hp, beam_window=beam)

    # Convert back to enmap
    ymap_smoothed = reproject.enmap_from_healpix(smoothed_hp, mask.shape, mask.wcs, rot=None)
    return ymap_smoothed

In [ ]:
ymap_filt.shape

In [ ]:
# Test on smaller patches
from numba import jit

@jit
def variance_map(imap, mask):
    '''Generates a variance map which calculates the variance of patches 100 x 100 pixels'''
    shape = imap.shape
    var_map = enmap.zeros(imap.shape, imap.wcs)
    patch_size = 100
    x,y = int(shape[1]/patch_size), int(shape[2]/patch_size)
    for i in range(x):
        for j in range(y):
            v = np.nanvar(mask[i*patch_size:(i+1)*patch_size, 
                               j*patch_size:(j+1)*patch_size] * \
                              imap[0][i*patch_size:(i+1)*patch_size, 
                                      j*patch_size:(j+1)*patch_size])
            
            var_map[i*patch_size:(i+1)*patch_size,
                    j*patch_size:(j+1)*patch_size] = v

    return var_map

def variance_map(imap, mask, minmax_list=None, centers_list=None, patch_size=100):
    '''Generates a variance map which calculates the variance of patches 100 x 100 pixels'''
    '''Min/max list is a list of [ra_min, ra_max, dec_min, dec_max] for every patch'''
    '''Centers_list is a list of centers for each patch. Cannot input both minmax and centers lists'''
    '''If Centers_list is not none, must also input patch size else 100x100 is assumed'''
    shape,wcs = imap.shape, imap.wcs
    var_map = enmap.zeros(imap.shape, imap.wcs)
    variances = []
    if minmax_list is not None:
        d=0
        for patch in minmax_list:
            ra_min,ra_max,dec_min,dec_max=np.deg2rad(patch[0]),np.deg2rad(patch[1]),np.deg2rad(patch[2]),np.deg2rad(patch[3])
            box   = np.array([[dec_min,ra_min],[dec_max,ra_max]]) # in radians
            omap  = imap.submap(box)
            omask = mask.submap(box)
            v = np.nanvar(omask*omap)
            variances.append(v)
            ll = enmap.sky2pix(shape,wcs,[dec_min,ra_max]).astype(int)
            ur = enmap.sky2pix(shape,wcs,[dec_max,ra_min]).astype(int)
            print(f'Patch {d}')
            print(ll)
            print(ur)
            var_map[ll[0]:ur[0],ll[1]:ur[1]] = v
            d+=1
            
#     elif centers_list is not None:
    
    else:
        x,y = int(shape[1]/patch_size), int(shape[2]/patch_size)
        for i in range(x):
            for j in range(y):
                v = np.nanvar(mask[i*patch_size:(i+1)*patch_size, 
                                   j*patch_size:(j+1)*patch_size] * \
                                  imap[0][i*patch_size:(i+1)*patch_size, 
                                          j*patch_size:(j+1)*patch_size])

                var_map[i*patch_size:(i+1)*patch_size,
                        j*patch_size:(j+1)*patch_size] = v

    return var_map

In [ ]:
eshow(var_map*mask/map_var_smooth, **{"colorbar":True, "downgrade":10})

In [ ]:
# The pixell routine is faster here but they don't completely agree
# ymap_smoothed = smooth_map(.35)
ymap_filt = smooth_map_pixell(.35)

In [ ]:
var_map = variance_map(ymap_filt,mask,minmax_list = [[5,10,-5,5],[280,30,-30,-20]])

In [ ]:
# Plot stamps comparing the two routines

plt.subplot(1, 3, 1)
plt.title("Pixell smoothing")
plt.imshow((ymap_filt*mask)[2000:2500,14000:14500],cmap='gray', vmax = 2e-10, vmin = -2e-10)

plt.subplot(1, 3, 2)
plt.title("Healpy smoothing")
plt.imshow((ymap_smoothed*mask)[0,2000:2500,14000:14500],cmap='gray', vmax = 2e-10, vmin = -2e-10)

plt.subplot(1, 3, 3)
plt.title("Difference")
plt.imshow(((ymap_smoothed-ymap_filt)*mask)[0,2000:2500,14000:14500],cmap='gray', vmax = 2e-10, vmin = -2e-10)

plt.colorbar()

In [ ]:
# Get variance of smoothed and un-smoothed maps

map_var = np.nanvar(ymap*mask)
map_var_smooth = np.nanvar(ymap_smoothed*mask)
map_var_smooth_pixell = np.nanvar(ymap_filt*mask)

print("The variance of the unsmoothed maps is {:.2e} \n \
  while the variance for the map smoothed with healpy is {:.2e} \n \
  and the variance for the map smoothed with pixell is {:.2e}".format(\
                                      map_var, map_var_smooth, map_var_smooth_pixell))

In [ ]:
# Test smoothing at different scales 
scales = [1/60, 5/60, 10/60, 20/60, 30/60, 1, 2]
variances = [] # collect total variance of map
ratios = [] # collcet variance of specific patch
for i in scales:
    ymap_smooth = smooth_map_pixell(i)
    map_var_smooth = np.nanvar(ymap_smooth*mask)
    variances.append(map_var_smooth)
    ratios.append(np.nanvar((ymap_smooth*mask)[2000:4000,12000:14000])/map_var_smooth)

In [ ]:
plt.plot(scales, variances)
plt.yscale('log')
plt.ylabel("Variance of Smoothed/Unsmoothed map")
plt.xlabel("Smoothing scale [degrees]")
plt.title("Variance of the whole map as a function of smoothing scale", loc='center',pad=20)
plt.show()

plt.plot(scales, ratios)
plt.title("Variance of a specific patch of the map as a function of smoothing scale", loc = "center",pad=20)
plt.ylabel("Variance of patch/whole map")
plt.xlabel("Smoothing scale [degrees]")

In [ ]:
# calculate the variance of patches of the smoothed map
var_map = variance_map(ymap_smoothed, mask)

In [ ]:
eshow(var_map*mask/map_var_smooth, **{"colorbar":True, "downgrade":10})